# Implementing Support Vector Machine using Sequential Minimization Optimization 

In [2]:
import pandas as pd
import numpy as np
#import prettytable
import sys
import warnings
from sklearn.datasets import load_breast_cancer

from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix
import time
from sklearn.utils import shuffle
warnings.filterwarnings('ignore')

# Defining class Support Vector Machine

In [3]:
class SupportVectorMachine():
    """
    construtor to initialize values for features and labels array 
    x and y respectively as well as lagrange multiplier α and bias b
    """
    def __init__ (self,x,y):
        self._x =x
        self._y=y
        self._alpha= np.mat(np.zeros((len(x),1)), dtype='float32')
        self._b =np.mat([[0]])
        
        i=0
        p=0
        while(i< max_iter):
#             if((i%10==0)&(i>0)):
#                 #progress bar after 10 iterations
#                 sys.stdout.write('='+ str(i))
#                 p=i%10
#                 sys.stdout.flush()

            if(self.perform_smo()==0): i+=1
            else: i=0
        
        self._w =self.calculate_w(np.nan_to_num(self._alpha), self._x,self._y)
    def perform_smo(self):
        """
        for this SMO algorthim we optimise taking random pairs of alpha
        such that they obey the constaint ∑i=1-l(yiαi)= 0. and at the same time maximise the 
        decision margin ||w||^2
        """
        numberAlphaPairsOptimized=0
        
        for i in range(0,len(self._x)):
            """
             Calculate the weight vector w= ∑i=1-l(αi yi xi)
             Ei= ∑j=1-l (αj yj K(xi, xj) +b) −yi, i= 1,2 
             Standard polynomial Kernelf fucntion K on S*S is of form, (a*b +r)^d 
             here ,d=1,r=0 , which gives
             Kij = K(xi ,xj)= (x1. x2) ,dot product 

            """    
           
            Ei =np.multiply(self._y , self._alpha).T* \
                self._x *self._x[i].T + self._b - self._y[i]
            #print(Ei)
            
            
            """
            KKT(i) =α(i){yi(〈w, xi〉+b)−1}
            if α=0 , corrreclty classified,
            α=c, misclassified or in the margin, c is slack parameter
            0<α<c , example is support vector
            
            """
            if(self.bool_alpha_violates_KKT(self._alpha[i],Ei)):
                
                j= self.select_second_alpha_to_optimise(i, len(self._x))
                Ej =np.multiply(self._y , self._alpha).T* \
                self._x *self._x[j].T + self._b - self._y[j]
                #print(Ej)
                alphaIold= self._alpha[i].copy()
                alphaJold= self._alpha[j].copy()
                
                bounds= self.bounds_alpha(self._alpha[i],self._alpha[j], self._y[i], self._y[j])
                """
                Calculate k= K11+K22−2∗K12
                """ 
                k= 2.0* self._x[i] * self._x[j].T \
                    -self._x[j] * self._x[i].T \
                    -self._x[j] * self._x[j].T
                
                if bounds[0]!=bounds[1] and k<0:
                    if self.optimise_alpha_pair(i,j, Ei, Ej, k, bounds, alphaIold, alphaJold):
                        numberAlphaPairsOptimized+=1
                    

                
        return numberAlphaPairsOptimized
    
    def optimise_alpha_pair( self, i,j, Ei, Ej, k, bounds, alphaIold, alphaJold):
        """
        upadte alpha 2 as - α2(new)= α2(old)+y2 * e(2)/k
        """
        flag=False
        self._alpha[j]-= self._y[j] * (Ei- Ej)/k
        self.clipAlpha_j(j,bounds)
        if(abs(self._alpha[j] -alphaJold)>= min_alpha_optimisation):
            self.optimiseAlphaIandAlphajOppDir(i,j,alphaJold)
            self.optimise_b(Ei,Ej, alphaIold,alphaJold,i,j)
            flag=True
        return flag
    
    def optimise_b(self, Ei, Ej, alphaIold, alphaJold, i,j):
        b1= self._b - Ei- self._y[i] *\
        (self._alpha[i]- alphaIold) * self._x[i] * self._x[i].T \
        - self._y[j] * (self._alpha[j]- alphaJold) * self._x[i] * self._x[j].T
        
        b2= self._b - Ej- self._y[i] *\
        (self._alpha[i]- alphaIold) * self._x[i] * self._x[j].T \
        - self._y[j] * (self._alpha[j]- alphaJold) * self._x[j] * self._x[j].T
        if(0< self._alpha[i]) and (c> self._alpha[i]): self._b=b1
        elif (0< self._alpha[j]) and (c> self._alpha[j]): self._b=b2
        else: self.b= (b1+b2)/2.0
            
            
    def bool_alpha_violates_KKT(self,alpha, E):
        c1=( alpha>0 and np.abs(E)< EPSILON)
        c2=(alpha<c and np.abs(E) > EPSILON)
        return c1 or c2
    
    def select_second_alpha_to_optimise(self, idxFirstAlpha, numRows):
        idxSecondAlpha =idxFirstAlpha
        while(idxFirstAlpha==idxSecondAlpha):
            idxSecondAlpha= int(np.random.uniform(0,numRows))
        
        return idxSecondAlpha
    
    def optimiseAlphaIandAlphajOppDir(self, i, j, alphaJold):
        """
        upadte alpha 1 as- α1(new)= α1(old)+y1y2(α2(old)−α2(new))
        """
        self._alpha[i]+= self._y[j] + self._y[i] * (alphaJold - self._alpha[j])
    
    def clipAlpha_j(self,j,bounds):
        if self._alpha[j]< bounds[0]: self._alpha[j]=bounds[0]
        if self._alpha[j] > bounds[1] : self._alpha[j]= bounds[1]
            
    
    def bounds_alpha(self,alphai , alphaj , yi , yj):
        bounds=[2]
        if(yi==yj):
            bounds.insert(0,max(0,alphaj+ alphai -c))
            bounds.insert(1, min(c, alphaj +alphai))
        else:
            bounds.insert(0, max(0, alphaj- alphai))
            bounds.insert(1, min(c,alphaj - alphai)+ c)
        return bounds
    
    def classification(self,x):
        classification = "class -1 negative"
#         return(x, self._w, self._b)
        wxb =(np.asarray(x).astype('float32',casting='unsafe') @ self._w + self._b).item(0,0)
        if(np.sign (wxb)==1):
            classification ="class +1 positive"
        return classification,wxb
    def get_alpha(self): return self._alpha
    def get_b(self): return self._b
    def get_w(self): return self._w
    
    def calculate_w( self, alpha, x,y):
        """
             Calculate the weight vector w= ∑i=1-l(αi*yi*xi)
        """
        w=np.zeros((np.shape(x)[1],1))
        #print(np.nan_to_num(alpha))
        for i in range(len(x)):
            w+= np.nan_to_num(np.multiply(y[i]* alpha[i] , x[i].T))
        return w

# Extracting data from text

In [ ]:
import pandas as pd
data=pd.read_csv('SVMdataTest.txt')
data.columns=['data']
data['x1']=data.data.str.split().str[0].astype(int)
data['x2']=data.data.str.split().str[1].astype(int)
data['y']=data.data.str.split().str[2].astype(int)
data=data.drop(columns={'data'})

# Training SVM on dataset

Defining model parameter

In [ ]:
max_iter=1000 #max attempts to optimise 
c=0.01 #slack parameter
min_alpha_optimisation=0.01 ## how much minimum pair of alpha shoud  optimise in order to consider alpha pair optimized
EPSILON= 0.0001 

In [ ]:
##shuffle train dataset
index = data.index
data=shuffle(data)
data.index=index

Fit SVM on linearly separble binary class data

In [96]:
X= data[['x1','x2']]
yArray=data['y']
#xArray=normalize(X, norm='max', axis=0)
xArray=np.asarray(X)
#xArray=xArray.astype('float32',casting='unsafe').round(7)
#yArray=[+1 if x==1 else -1 for x in yArray]

svm= SupportVectorMachine(np.mat(xArray[0:200]), np.mat(yArray[0:200]).transpose())



Display support vectors , class labels ,alpha values and weights learned by model for x1 and x2

In [97]:
alpha=svm.get_alpha()
def display_info_tables():
    svTable= prettytable.PrettyTable(['Support Vector','label','alpha'])
    for i in range(len(xArray[:200])):
        if ((alpha[i]>0.0) and (alpha[i]!=c)):
            svTable.add_row([xArray[i], yArray[i], alpha[i].item()])
    print(svTable)
    wbTable=prettytable.PrettyTable(['wT','b'])
    wbTable.add_row([svm.get_w().T, svm.get_b()])
    print(wbTable)
display_info_tables()
print('close window to proceed')

+----------------+-------+------------------------+
| Support Vector | label |         alpha          |
+----------------+-------+------------------------+
|   [270 300]    |   1   | 3.370608965269639e-06  |
|   [165 195]    |   1   | 4.150081440457143e-05  |
|   [235 211]    |   -1  | 0.0002837955253198743  |
|   [111  90]    |   -1  | 0.00042558982386253774 |
|    [66 93]     |   1   | 0.00014853286847937852 |
|   [147 122]    |   -1  | 2.794317333609797e-05  |
|   [147 175]    |   1   | 0.0008821747032925487  |
|   [275 241]    |   -1  | 9.002593287732452e-05  |
|   [182 156]    |   -1  | 0.0003094676067121327  |
|    [79 50]     |   -1  | 6.777705857530236e-05  |
|    [47 75]     |   1   | 0.0004292913363315165  |
+----------------+-------+------------------------+
+-----------------------------+-------+
|              wT             |   b   |
+-----------------------------+-------+
| [[-0.03705745  0.03453977]] | [[0]] |
+-----------------------------+-------+
close window to proc

# weight of both x1 and x2 are -0.04360495 

Test code on remaining data

In [98]:
#### test data #####
act= yArray[200:]

pred=[]
###### getting predictions #######
for k in xArray[200:]:  
    t=svm.classification(k)
    #print(t[1])
    if(t[0]=='class +1 positive'):
        pred.append(1)
    else:
        pred.append(-1)

##### saving parameter values and output in datframe ######
t0=confusion_matrix(act,pred)
t0

array([[33,  0],
       [ 0, 32]], dtype=int64)

# Additional testing of SVMwSMO on sklearn breast cancer data

In [85]:
data = load_breast_cancer()
max_iter=5000 #max attempts to optimise 
c=100 #slack parameter
min_alpha_optimisation=0.000001
EPSILON= 0.001

df=pd.DataFrame(data['data'], columns=data['feature_names'])
df1=pd.DataFrame(data['target'], columns=['target'])
df=pd.concat([df,df1],axis=1)

index = df.index
df=shuffle(df)
df.index=index

X=df[['mean concave points',
 'area error',
 'worst concave points',
 'worst radius',
 'mean concavity',
 'worst perimeter',
 'mean area']]
# scaler = StandardScaler()
# scaler.fit(X)
# X=scaler.transform(X)

# pca = PCA()
# pca.fit(X)
# x_new = pca.transform(X) 
    
# xArray, yArray= x_new[0:400], (df['target'].loc[0:399])
yArray=df['target']
xArray=normalize(X, norm='max', axis=0)

xArray=xArray.astype('float32',casting='unsafe').round(7)
yArray=[+1 if x==1 else -1 for x in yArray]

svm= SupportVectorMachine(np.mat(xArray[0:400]), np.mat(yArray[0:400]).transpose())



Trained on 400 rows and tested on 200 rows, confusion matrix looks like

In [86]:
#### test data #####
act= yArray[400:]

pred=[]
###### getting predictions #######
for k in xArray[400:]:  
    t=svm.classification(k)
    #print(t[1])
    if(t[0]=='class +1 positive'):
        pred.append(1)
    else:
        pred.append(-1)

##### saving parameter values and output in datframe ######
t0=confusion_matrix(act,pred)
t0

array([[ 0, 54],
       [28, 87]], dtype=int64)

# Grid on hyperparmeters  epsilon, slack parameter and min_alpha_opt

In [316]:
#### test data #####
act= np.asarray(df['target'].loc[200:])
act=[+1 if x==1 else -1 for x in act]

##### parameter sarch ######
para= pd.DataFrame(columns= ['max_iter','min_alpha_optimization','epsilon','slack_parameter','TN', 'FP', 'FN', 'TP', 'iter_time'])
i=0
#### grid search on parameters
for max_iter in [1000, 2000, 10000]:
    for min_alpha_optimisation in [1e-1 , 1e-2, 1e-3 , 1e-4, 1e-5, 1e-6]:
        for EPSILON in [1,0.1, 0.01, 0.001, 1e-4, 1e-5]:
            for c in [0.1, 0.01, 1, 10]:
                sys.stdout.write('='+ str(round(100*(i/432))) +'%=')
                sys.stdout.flush()
                st=time.time()
                svm= SupportVectorMachine(np.mat(xArray), np.mat(yArray).transpose())
                en= time.time()
                pred=[]
                ###### getting predictions #######
                for k in range(200, df.shape[0]):  
                    t=svm.classification(df[['worst perimeter','worst concave points','mean concave points','worst area','worst radius','mean texture',
      'worst compactness']])
                    if(t[0]=='class +1 positive'):
                        pred.append(1)
                    else:
                        pred.append(-1)
                
                ##### saving parameter values and output in datframe ######
                t0=confusion_matrix(act,pred)
                para.loc[i]= [max_iter, min_alpha_optimisation, EPSILON, c, t0[0][0], t0[0][1], t0[1][0], t0[1][1],round(en-st)]
                #print(para)
                i+=1

para.to_csv('gridSearchSVMParameters.csv')
 

=0%==0%==0%==1%==1%==1%==1%==2%==2%==2%==2%==3%==3%==3%==3%==3%==4%==4%==4%==4%==5%==5%==5%==5%==6%==6%==6%==6%==6%==7%==7%==7%==7%==8%==8%==8%==8%==9%==9%==9%==9%==9%==10%==10%==10%==10%==11%==11%==11%==11%==12%==12%==12%==12%==12%==13%==13%==13%==13%==14%==14%==14%==14%==15%==15%==15%==15%==16%==16%==16%==16%==16%==17%==17%==17%==17%==18%==18%==18%==18%==19%==19%==19%==19%==19%==20%==20%==20%==20%==21%==21%==21%==21%==22%==22%==22%==22%==22%==23%==23%==23%==23%==24%==24%==24%==24%==25%==25%==25%==25%==25%==26%==26%==26%==26%==27%==27%==27%==27%==28%==28%==28%==28%==28%==29%==29%==29%==29%==30%==30%==30%==30%==31%==31%==31%==31%==31%==32%==32%==32%==32%==33%==33%==33%==33%==34%==34%==34%==34%==34%==35%==35%==35%==35%==36%==36%==36%==36%==37%==37%==37%==37%==38%==38%==38%==38%==38%==39%==39%==39%==39%==40%==40%==40%==40%==41%==41%==41%==41%==41%==42%==42%==42%==42%==43%==43%==43%==43%==44%==44%==44%==44%==44%==45%==45%==45%==45%==46%==46%==46%==46%==47%==47%==47%==47%==47%==48%==48%==4

KeyboardInterrupt: 

In [317]:
para.to_csv('gridSearchSVMParameters.csv')
para['accuracy']=   round(100*((para['TP']+ para['TN'])/( para['TP']+ para['FP'] + para['TN']+ para['FN'])))

Selecting the most important features from breast cancer data coming from random forest regression and only using those to train SVM algorthim

# Trying sklearn SVM to compare the results

In [342]:
from sklearn import svm
df=pd.DataFrame(data['data'], columns=data['feature_names'])
df1=pd.DataFrame(data['target'], columns=['target'])
df=pd.concat([df,df1],axis=1)
from sklearn.utils import shuffle
index = df.index
df=shuffle(df)
df.index=index
clf = svm.SVC()
X=normalize(data['data'][0:400], norm='max', axis=0)
Y=data['target'][0:400]
Y=[+1 if x==1 else -1 for x in Y]
clf.fit(X,Y )
x=normalize(data['data'][400:], norm='max', axis=0)

pred=clf.predict(x)

act= np.asarray(df['target'].loc[400:])
act=[+1 if x==1 else -1 for x in act]
confusion_matrix(act,pred)

array([[10, 55],
       [31, 73]], dtype=int64)

# Trying Reandom Forest Regressor to get the best set of features

In [332]:
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier

data = load_breast_cancer()
df=pd.DataFrame(data['data'], columns=data['feature_names'])
df1=pd.DataFrame(data['target'], columns=['target'])
df=pd.concat([df,df1],axis=1)

X, y = make_classification(n_samples=1000, n_features=4,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)

regr = RandomForestClassifier(max_depth=2, random_state=0)
regr.fit(df.loc[:, :'worst fractal dimension'], df['target'])


t=pd.DataFrame(columns={'imp','features'})
t['imp']=regr.feature_importances_
t['features']=df.columns[:-1]

In [333]:
t.sort_values(by='imp',ascending=False)[0:7].features.to_list()

['mean concave points',
 'area error',
 'worst concave points',
 'worst radius',
 'mean concavity',
 'worst perimeter',
 'mean area']

USe the above features to train the SVM as dimentionality poses threat to convergence of SV algorthim